In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [3]:
test = unpickle("/home/pirl/cifar-100-python/test")
train = unpickle("/home/pirl/cifar-100-python/train")

In [4]:
train.keys()

dict_keys([b'filenames', b'batch_label', b'fine_labels', b'coarse_labels', b'data'])

In [5]:
len(train[b'data'][0])

3072

In [6]:
train_x = np.transpose(train[b'data'].reshape(-1,3,32,32),(0,2,3,1))
train_y = train[b'fine_labels']
test_x = np.transpose(test[b'data'].reshape(-1,3,32,32),(0,2,3,1))
test_y = test[b'fine_labels']

In [7]:
train_x.shape

(50000, 32, 32, 3)

In [15]:
np.array(test_y).shape

(10000,)

In [9]:
learning_rate = 1e-3
num_epochs = 100
batch_size = 100
num_display = 100

![screensh](https://cdn-images-1.medium.com/max/1600/1*M5NIelQC33eN6KjwZRccoQ.png)

In [19]:
def conv(X,filters,name,strides=(1,1)):
    outs = tf.layers.conv2d(X, filters, 3, padding='same', name=name, reuse=tf.AUTO_REUSE)
    outs = bn(outs)
    return outs

In [18]:
def bn(X):
    with tf.variable_scope(name):
        batch_mean, batch_var = tf.nn.moments(X,[0])
    return tf.nn.batch_normalization(X,batch_mean,batch_var,0,1,1e-3,name=name)

In [11]:
def convolutional_block(X,filters,name):
    with tf.variable_scope(name):
        outs = conv(X,filters,name)
        outs = tf.nn.relu(outs)
        outs = conv(outs,filters,name)
        outs = tf.nn.relu(X + outs)
    return outs

In [12]:
# def identity_block(X,filters,name):
#     with tf.variable_scope(name):
#         outs = tf.layers.conv2d(X, filters, 3,(2,2),padding='same', name=name, reuse=tf.AUTO_REUSE)
#         outs = tf.layers.conv2d(outs, filters*2,3,padding='same', name=name, reuse=tf.AUTO_REUSE)
#     return outs

In [20]:
def skip_connection(X,filters,name):
    with tf.variable_scope(name):
        sc = conv(X,filters,name,strides=(2,2))
        sc = conv(sc,filters*2,name)
    return sc

In [21]:
def ResNet(X,filter_list,filter_iters):
    outs = tf.layers.conv2d(X,filter_list[0],1,padding='same',name=first, reuse=False)
    for filters in filter_list:
        for i in range(filter_iters[0]):
            outs = convolutional_block(outs,filters,'conv-{0}-{1}'.format(filters,i))
        if filters == filter_list[-1]:
            outs = tf.nn.avg_pool(outs,)
        else:
            outs = skip_connection(outs)
    
    outs = 

SyntaxError: invalid syntax (<ipython-input-21-b033c19cb839>, line 7)

In [17]:
type(train_x)

numpy.ndarray

In [18]:
with tf.Session() as sess:
    with tf.variable_scope('first'):
        tf.get_variable(tf.layers.conv2d(train_x,32,1,padding='same',name='conv'))
        sess.run(['conv 

AttributeError: 'tuple' object has no attribute 'ndims'

In [22]:
def get_model(X, by, is_reuse):
#     X = tf.expand_dims(X, axis=3) # (None, 32, 32, 3)
    
    with tf.variable_scope('first'):
        outs = tf.layers.conv2d(X, 128, 3, padding='same', name='conv1', reuse=tf.AUTO_REUSE) # (None, 32, 32, 128)
        outs = tf.nn.relu(outs)
        outs = tf.layers.max_pooling2d(outs, 2, 2) # (None, 14, 14, 128)
    with tf.variable_scope('second'):
        outs = tf.layers.conv2d(outs, 256, 3, padding='same', name='conv2')
        outs = tf.nn.relu(outs)
        outs = tf.layers.max_pooling2d(outs, 2, 2) # (None, 7, 7, 256)
    with tf.variable_scope('third'):
        outs = tf.layers.conv2d(outs, 64, 3, padding='same', name='conv3')
        outs = tf.nn.relu(outs)
        outs = tf.layers.max_pooling2d(outs, 2, 2) # (None, 3, 3, 64)
    
    outs = tf.reshape(outs, (-1, outs.shape[1]*outs.shape[2]*outs.shape[3]))
    
    with tf.variable_scope('dense'):
        outs = tf.layers.dense(outs,128, name='dense1')
        outs = tf.nn.relu(outs)
        outs = tf.layers.dense(outs, 100, name='dense2')
    
    one_hot = tf.one_hot(by, 100)
    
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=outs, 
                                                      labels=one_hot)
    loss = tf.reduce_mean(loss)
    opt = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
    
    preds = tf.cast(tf.argmax(tf.nn.softmax(outs), axis=1), tf.int32)
    acc = tf.reduce_mean(tf.cast(tf.equal(by, preds), tf.float32))
    saver = tf.train.Saver(
            tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,'second')
            +tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,'thrid')
            +tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,'dense')
    )
    
    init = tf.global_variables_initializer()
    return {
        'loss': loss,
        'opt': opt,
        'preds': preds,
        'acc': acc,
        'init': init,
        'saver': saver
    }

In [19]:
X = tf.placeholder(tf.float32, shape=(None, 32, 32, 3))
by = tf.placeholder(tf.int32)

model = get_model(X, by, True)

In [20]:
with tf.Session() as sess:
    sess.run(model['init'])
    for ind_epoch in range(0, num_epochs):
        print('Current iteration {}'.format(ind_epoch + 1))
        
        for ind_ in range(0, int(50000 / batch_size)):
            batch_X = train_x[ind_*batch_size:(ind_+1)*batch_size]
            batch_by = train_y[ind_*batch_size:(ind_+1)*batch_size]
            
            _, cur_loss, cur_acc = sess.run([model['opt'], model['loss'], 
                                             model['acc']],
                                             feed_dict={X: batch_X, by: batch_by}
                                           )
            if ind_ % num_display == 0:
                print('loss {0:.4f} acc {1:.4f}'.format(cur_loss, cur_acc))
    model['saver'].save(sess, './ours.ckpt')

Current iteration 1
loss 39.4301 acc 0.0100
loss 4.6125 acc 0.0100
loss 4.5674 acc 0.0200
loss 4.5118 acc 0.0500
loss 4.5217 acc 0.0300
Current iteration 2
loss 4.4347 acc 0.0500
loss 4.5030 acc 0.0300
loss 4.4809 acc 0.0400
loss 4.4360 acc 0.0300
loss 4.4270 acc 0.0300
Current iteration 3
loss 4.3155 acc 0.0800
loss 4.4205 acc 0.0300
loss 4.3932 acc 0.0300
loss 4.3387 acc 0.0300
loss 4.2994 acc 0.0700
Current iteration 4
loss 4.2273 acc 0.0700
loss 4.3434 acc 0.0300
loss 4.3079 acc 0.0500
loss 4.2153 acc 0.0500
loss 4.1431 acc 0.1100
Current iteration 5
loss 4.1119 acc 0.0900
loss 4.2658 acc 0.0400
loss 4.1939 acc 0.1000
loss 4.1040 acc 0.0500
loss 4.0110 acc 0.1100
Current iteration 6
loss 3.9600 acc 0.1400
loss 4.1927 acc 0.0500
loss 4.0645 acc 0.1300
loss 3.9704 acc 0.0500
loss 3.8890 acc 0.1500
Current iteration 7
loss 3.8225 acc 0.1800
loss 4.1359 acc 0.0600
loss 3.9807 acc 0.1400
loss 3.8796 acc 0.0800
loss 3.7667 acc 0.1500
Current iteration 8
loss 3.7393 acc 0.2000
loss 4.0377

loss 1.7797 acc 0.5000
loss 1.5335 acc 0.6100
loss 1.7259 acc 0.6500
loss 2.0857 acc 0.4400
Current iteration 62
loss 1.7969 acc 0.5500
loss 1.7598 acc 0.5100
loss 1.5213 acc 0.5900
loss 1.7253 acc 0.6300
loss 2.0472 acc 0.4400
Current iteration 63
loss 1.7639 acc 0.5600
loss 1.7420 acc 0.5400
loss 1.4980 acc 0.6200
loss 1.6795 acc 0.6600
loss 2.0380 acc 0.4500
Current iteration 64
loss 1.7279 acc 0.5400
loss 1.7079 acc 0.5300
loss 1.4893 acc 0.5900
loss 1.7153 acc 0.6200
loss 2.0280 acc 0.4500
Current iteration 65
loss 1.7008 acc 0.5700
loss 1.7068 acc 0.5000
loss 1.4625 acc 0.5900
loss 1.6560 acc 0.6500
loss 1.9665 acc 0.4600
Current iteration 66
loss 1.6878 acc 0.5500
loss 1.6834 acc 0.5400
loss 1.4216 acc 0.6200
loss 1.6654 acc 0.6400
loss 1.9621 acc 0.4800
Current iteration 67
loss 1.6534 acc 0.6000
loss 1.6833 acc 0.5700
loss 1.4021 acc 0.6100
loss 1.6106 acc 0.6600
loss 1.9211 acc 0.4600
Current iteration 68
loss 1.6243 acc 0.5800
loss 1.6326 acc 0.5700
loss 1.3776 acc 0.6100
lo